<a href="https://colab.research.google.com/github/Carleto04/btcPred/blob/main/btc_pred_LSTM_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install finnhub-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
## add code for Real Time data from twitter and BTC prices
#--------------------------------------------------------------------#
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import numpy as np
import datetime as dt
import finnhub
import re
from textblob import TextBlob
from sklearn import preprocessing
import datetime
np.random.seed(12345)

# Data download and cleaning
# ==============================================================================
# ....... data from BTC ....... #
# BTC prices data
api_key = 'cakpfr2ad3ier73mcivg'
stock = 'SQ'
resolution = 'D'
end_date = datetime.datetime(2021, 3, 12, 0 ,0)
start_date = end_date - dt.timedelta(days=365)
end = int(end_date.timestamp())
start = int(start_date.timestamp())
finnhub_client = finnhub.Client(api_key=api_key)
result = finnhub_client.crypto_candles('BINANCE:BTCUSDT', 'D', start, end)

# Transform collected data into a dataframe
btcDF = pd.DataFrame(result)

# pass values to usable format
btcDF['Timestamp'] = pd.to_datetime(btcDF['t'], unit='s')
btcDF = btcDF.set_index(pd.DatetimeIndex(btcDF['Timestamp']))
btcDF = btcDF.drop(['Timestamp', 't', 's', 'h', 'l', 'o'], axis=1)
btcDF.columns = ['Close_Price', 'Volume_BTC']
btcDF.head()

,Close_Price,Volume_BTC
Timestamp,,
2020-03-12,4800.00,261505.608653
2020-03-13,5578.60,402201.673764
2020-03-14,5172.06,136910.135974
2020-03-15,5361.30,139916.146534
2020-03-16,5028.97,227276.922760


In [ ]:
# import required modules
import zipfile
import pandas as pd
 
# read the dataset using the compression zip
df = pd.read_csv('test.zip',compression='zip')
 
# display dataset
print(df.head())

In [4]:
# ....... data from twitter ....... #

# Sentiment analyisis class
# ==============================================================================
notclean = pd.read_csv('sample_data/Bitcoin_tweets.csv')
#columns=['user_name', 'user_location', 'user_description', 'user_created',
#       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
#       'date', 'text', 'hashtags', 'source', 'is_retweet']
notclean =notclean.drop(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'hashtags', 'source', 'is_retweet'], axis=1)
txt = notclean.text

clean_txt = []
for i in range(len(txt)):
    clean_txt.append(' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", txt[i]).split()))

FileNotFoundError: ignored

In [ ]:
polarity_score = []
sensitivity_score = []
for e in clean_txt:
    polarity_score.append(TextBlob(e).sentiment.polarity)
    sensitivity_score.append(TextBlob(e).sentiment.subjectivity)

notclean['Polarity'] = polarity_score
notclean['Sensitivity'] = sensitivity_score

In [ ]:
notclean.info()

In [ ]:
## pass dateTime values to usable format
notclean.date = pd.to_datetime(notclean.date, errors='coerce')

## use of floor mehod, which will round all the times to the hour
notclean['DateTime'] = notclean['date'].dt.floor('h')

## Create a new feature ‘Tweet vol’ by aggregating and grouping all the tweets by hour.
vdf = notclean.groupby(pd.Grouper(key='date',freq='H')).size().reset_index(name='tweet_vol')
vdf.head()

In [ ]:
## pass dateTime values to usable format and set them to index
vdf.index = pd.to_datetime(vdf.index)
vdf=vdf.set_index('date')
vdf.info()

In [ ]:
print(vdf.tail())
print(vdf.head())

In [ ]:
# set DateTime as index
notclean.index = pd.to_datetime(notclean.index)
notclean =notclean.drop('text', axis=1)
notclean.info()

In [ ]:
vdf['tweet_vol'] =vdf['tweet_vol'].astype(float)
vdf.info()

In [ ]:
# Use of the important information in a single new dataframe
df = notclean.groupby('DateTime').agg(lambda x: x.mean())
df['Tweet_vol'] = vdf['tweet_vol']
df.head()

In [ ]:
df = df.drop('date', axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
# ....... joined data ....... #

# merge all data
Final_df = pd.merge(df,btcDF, how='inner',left_index=True, right_index=True)
Final_df.info()

In [ ]:
Final_df

In [ ]:
Final_df.to_csv('sample_data/Sentiment.csv')

In [ ]:
cor = Final_df.corr()
print(cor)

In [ ]:
sns.set(style="white")
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
ax =sns.heatmap(cor, cmap=cmap, vmax=1, center=0,
           square=True, linewidths=.5, cbar_kws={"shrink": .7})
plt.show()

In [ ]:
Final_df['time']=Final_df.index.to_series().apply(lambda x: x.strftime("%X"))
Final_df.head()

In [ ]:
#---------------- LSTM Prep ------------------------#

df = Final_df
df = df.drop(['Open','High', 'Low', 'Volume_Dollar'], axis=1)
df = df[['Close_Price', 'Polarity', 'Sensitivity','Tweet_vol','Volume_BTC']]
df.head()
df.info()

In [ ]:
# ....... understand correlations ....... #

cor = df.corr()
sns.set(style="white")
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
ax =sns.heatmap(cor, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .7})
plt.show()

In [ ]:
# ....... LSTM Model ....... #
from math import sqrt
from numpy import concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = DataFrame(data)
  cols, names = list(), list()
# input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
      names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
  agg = concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(inplace=True)
  return agg

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df.values)

n_hours = 3 #adding 3 hours lags creating number of observations 
n_features = 5 #Features in the dataset.
n_obs = n_hours*n_features

reframed = series_to_supervised(scaled, n_hours, 1)
reframed.drop(reframed.columns[-4], axis=1)
print(reframed.head())

values = reframed.values
n_train_hours = 200
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
train.shape

# split into input and outputs
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# design network
model = Sequential()
model.add(LSTM(5, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=6, validation_data=(test_X, test_y), verbose=2, shuffle=False,validation_split=0.2)

# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours* n_features,))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -4:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -4:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
mse = (mean_squared_error(inv_y, inv_yhat))
print('Test MSE: %.3f' % mse)
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

plt.plot(inv_y, label='Real')
plt.plot(inv_yhat, label='Predicted')
plt.title('Real v Predicted Close_Price')
plt.ylabel('Price ($)')
plt.xlabel('epochs (Hr)')
plt.show()